In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/news/
!ls

Mounted at /content/drive
/content/drive/MyDrive/news
 ColabModels		      news_embeddings.ipynb
 data			      news_preprocess.ipynb
 DatasetNews.ipynb	      predict_range_using_news.ipynb
'LLM_News_v1 (1) (1).ipynb'   results


In [ ]:
import pandas as pd
import numpy as np
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/news/data/test_embed_news.csv')
df.head()

,date,pegasus_summ
0,2025-02-25 00:00:00,BlockDAG (BDAG) is a cryptocurrency that uses ...
1,2025-02-25 00:00:00,"Bitcoin has dropped below $90,000 for the firs..."
2,2025-02-25 00:33:00,A proposal in the Montana House of Representat...
3,2025-02-25 00:35:00,Binance Co-founder Changpeng Zhao has shared h...
4,2025-02-25 00:38:00,Binance Co-founder Changpeng Zhao has revealed...


In [ ]:
df = df.tail(3195)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18114 entries, 0 to 18113
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          18114 non-null  object
 1   pegasus_summ  18114 non-null  object
dtypes: object(2)
memory usage: 283.2+ KB


In [ ]:
btc_news_summary = df[df['asset_symbols'].str.contains(r'\bBTC\b', case=False, na=False)]

In [ ]:
btc_news_summary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18114 entries, 1 to 241648
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             18114 non-null  object
 1   releasedAt     18114 non-null  object
 2   title          18114 non-null  object
 3   subtitle       18106 non-null  object
 4   sourceName     18114 non-null  object
 5   sourceUrl      18114 non-null  object
 6   source         18114 non-null  object
 7   slug           18114 non-null  object
 8   asset_names    18109 non-null  object
 9   asset_slugs    18109 non-null  object
 10  asset_symbols  18114 non-null  object
 11  content        18114 non-null  object
 12  pegasus_summ   18114 non-null  object
dtypes: object(13)
memory usage: 1.9+ MB


In [ ]:
for i, r in enumerate(df['content'].isnull()):
  if r:
    print(i)

323
6699
8038
9731
11236
11240
14851
14852
15853
15959
15960
15967
15968
15987
15991
15993
16054
16061
16069
16073
16953
20167
20230
25786
27783
29737
29932
30155
34158
34159
34160
34208
34214
34226
34236
34269
34288
34296
34315
34321
34339
34419
34420
34421
34435
34502
34504
34554
34584
34645
34646
34651
34678
34679
34683
34691
36043
38649
38650
38663
38710
38713
38715
38727
38728
38734
38753
38758
38814
39134
39147
40481
40610
40616
40735
41248
41257
41260
41261
41262
41264
41266
41279
41280
41290
43094
43095
43096
43751
43788
44081
50614
52292
52307
52316
52320
52321
52368
52370
52379
52383
52402
52403
52428
52429
52443
52445
52486
52490
52492
54455
55232
55237
55252
55276
55609
57487
57491
57492
57498
57502
59986
59992
60532
64405
64406
64416
64418
64897
65490
65629
66366
66391
66768
66776
66788
66796
66797
66824
66839
66849
66850
66853
66868
66873
66878
66891
68173
68789
70292
74988
76714
76921
76994
81597
86544
90483
90484
91559
91566
91874
91877
92924
92927
92929
92934
92935
929

In [ ]:
print(df['pegasus_summ'][209010])

In an interview with Oprah Winfrey, former New York City Mayor Rudy Giuliani said, "I don't think there's anybody in New York City who doesn't think that there's something wrong with the city of New York." He added, "I don't think there


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/TimesNet/Time-Series-Library/dataset/my_data/fixed_data_btc_1min.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142561 entries, 0 to 142560
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              142561 non-null  object 
 1   open              142561 non-null  float64
 2   high              142561 non-null  float64
 3   low               142561 non-null  float64
 4   close             142561 non-null  float64
 5   volume            142561 non-null  float64
 6   quoteAssetVolume  142561 non-null  float64
 7   numberOfTrades    142561 non-null  int64  
 8   takerBuyBaseVol   142561 non-null  float64
 9   takerBuyQuoteVol  142561 non-null  float64
 10  ignore            142561 non-null  int64  
 11  pre_return        142561 non-null  float64
 12  return            142561 non-null  float64
dtypes: float64(10), int64(2), object(1)
memory usage: 14.1+ MB


In [ ]:
df2 = df2[df2['date'].isin(btc_news_summary['releasedAt'])]

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3291 entries, 34560 to 141755
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              3291 non-null   object 
 1   open              3291 non-null   float64
 2   high              3291 non-null   float64
 3   low               3291 non-null   float64
 4   close             3291 non-null   float64
 5   volume            3291 non-null   float64
 6   quoteAssetVolume  3291 non-null   float64
 7   numberOfTrades    3291 non-null   int64  
 8   takerBuyBaseVol   3291 non-null   float64
 9   takerBuyQuoteVol  3291 non-null   float64
 10  ignore            3291 non-null   int64  
 11  pre_return        3291 non-null   float64
 12  return            3291 non-null   float64
dtypes: float64(10), int64(2), object(1)
memory usage: 360.0+ KB


In [ ]:
df.to_csv('/content/drive/MyDrive/OHLCV/Time-Series-Library/dataset/my_data/test_btc_ohlcv_3195.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
model.eval()

# Disable gradient computation for efficiency
torch.set_grad_enabled(False)

In [ ]:
def get_summary_embeddings(summaries, model, tokenizer, method='cls', batch_size=16):
    """
    Extract embeddings from summaries in freeze mode

    Args:
        summaries: List of summary texts
        model: FinBERT model
        tokenizer: FinBERT tokenizer
        method: 'cls', 'mean_pool', or 'max_pool'
        batch_size: Number of summaries to process at once

    Returns:
        numpy array of embeddings
    """
    all_embeddings = []

    for i in range(0, len(summaries), batch_size):
        batch_summaries = summaries[i:i + batch_size]

        inputs = tokenizer(
            batch_summaries,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )

        with torch.no_grad():
            outputs = model(**inputs)

            if method == 'cls':
                embeddings = outputs.last_hidden_state[:, 0, :]

            elif method == 'mean_pool':
                embeddings = mean_pooling(outputs, inputs['attention_mask'])

            elif method == 'max_pool':
                embeddings = torch.max(outputs.last_hidden_state, dim=1)[0]

        all_embeddings.append(embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

def mean_pooling(model_output, attention_mask):
    """Mean pooling considering attention mask"""
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
summaries = df['pegasus_summ'].tolist()

embeddings = get_summary_embeddings(summaries, model, tokenizer, method='cls')

print(f"Embeddings shape: {embeddings.shape}")
print(f"Number of summaries: {len(summaries)}")
print(f"Embedding dimension: {embeddings.shape[1]}")

Embeddings shape: (3195, 768)
Number of summaries: 3195
Embedding dimension: 768


In [ ]:
import pickle

def save_embeddings(embeddings, save_path):
    data = {
        'embeddings': embeddings,
        'embedding_shape': embeddings.shape,
    }

    with open(save_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"Embeddings saved to {save_path}")
    print(f"Embedding shape: {embeddings.shape}")

In [ ]:
save_embeddings(embeddings, '/content/drive/MyDrive/Final_Block/test_news_embeddings.pkl')

Embeddings saved to /content/drive/MyDrive/Final_Block/test_news_embeddings.pkl
Embedding shape: (3195, 768)


In [ ]:
!pwd

/content/drive/MyDrive/news
